In [2]:
pip install pandas-datareader

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas_datareader.data as web
import numpy as np
import pandas as pd

In [10]:
stk_tickers =["MSFT","IBM","GOOGL"]
ccy_tickers = ["DEXJPUS","DEXUSUK"]
idx_tickers = ["SP500","DJIA","VIXCLS"]

In [12]:
stk_data = web.DataReader(stk_tickers,"yahoo")
ccy_data = web.DataReader(ccy_tickers,"fred")
idx_data = web.DataReader(idx_tickers,"fred")

In [13]:
stk_data

Attributes   Adj Close                                Close              \
Symbols           MSFT         IBM        GOOGL        MSFT         IBM   
Date                                                                      
2017-02-21   60.200428  136.453751   849.270020   64.489998  172.332703   
2017-02-22   60.079079  137.127441   851.359985   64.360001  173.183563   
2017-02-23   60.321785  137.505936   851.000000   64.620003  173.661575   
2017-02-24   60.321785  137.278839   847.809998   64.620003  173.374756   
2017-02-27   59.957729  135.802719   849.669983   64.230003  171.510513   
...                ...         ...          ...         ...         ...   
2022-02-11  294.431213  132.690002  2685.649902  295.040009  132.690002   
2022-02-14  294.391296  130.149994  2710.520020  295.000000  130.149994   
2022-02-15  299.850006  129.940002  2732.169922  300.470001  129.940002   
2022-02-16  299.500000  129.179993  2754.760010  299.500000  129.179993   
2022-02-17  290.730011  124.970001  2650.780029  290.730011  124.970001   

Attributes                     High                                  Low  \
Symbols           GOOGL        MSFT         IBM        GOOGL        MSFT   
Date                                                                       
2017-02-21   849.270020   64.949997  172.839386   852.200012   64.449997   
2017-02-22   851.359985   64.389999  173.365204   853.789978   64.050003   
2017-02-23   851.000000   64.730003  174.474182   852.619995   64.190002   
2017-02-24   847.809998   64.800003  173.508606   848.359985   64.139999   
2017-02-27   849.669983   64.540001  173.279160   850.669983   64.050003   
...                 ...         ...         ...          ...         ...   
2022-02-11  2685.649902  304.290009  134.710007  2783.129883  294.220001   
2022-02-14  2710.520020  296.760010  132.649994  2726.000000  291.350006   
2022-02-15  2732.169922  300.799988  131.679993  2762.169922  297.019989   
2022-02-16  2754.760010  300.869995  130.440002  2761.719971  293.679993   
2022-02-17  2650.780029  296.799988  128.500000  2742.270020  290.000000   

Attributes                                 Open                           \
Symbols            IBM        GOOGL        MSFT         IBM        GOOGL   
Date                                                                       
2017-02-21  171.242828   846.549988   64.610001  172.657745   847.989990   
2017-02-22  172.170166   846.710022   64.330002  172.170166   848.000000   
2017-02-23  172.963669   842.500000   64.419998  173.996170   851.080017   
2017-02-24  171.978973   842.960022   64.529999  172.284897   847.650024   
2017-02-27  171.395798   843.010010   64.540001  173.221802   844.950012   
...                ...          ...         ...         ...          ...   
2022-02-11  132.380005  2668.000000  303.190002  133.899994  2772.000000   
2022-02-14  129.070007  2665.129883  293.769989  132.589996  2665.129883   
2022-02-15  129.610001  2716.429932  300.010010  130.639999  2751.409912   
2022-02-16  128.259995  2698.479980  298.369995  129.449997  2732.929932   
2022-02-17  124.849998  2649.570068  296.359985  128.050003  2724.860107   

Attributes      Volume                        
Symbols           MSFT        IBM      GOOGL  
Date                                          
2017-02-21  20655900.0  4686394.0  1260300.0  
2017-02-22  19292700.0  3159443.0  1224400.0  
2017-02-23  20273100.0  2387495.0  1386700.0  
2017-02-24  21796800.0  3451905.0  1346200.0  
2017-02-27  15871500.0  3861100.0  1010300.0  
...                ...        ...        ...  
2022-02-11  39143900.0  4175600.0  1994200.0  
2022-02-14  36339400.0  5344300.0  1713900.0  
2022-02-15  27379500.0  4395700.0  1335600.0  
2022-02-16  29982100.0  4875600.0  1251100.0  
2022-02-17  32384200.0  6794800.0  1941600.0  

[1259 rows x 18 columns]

In [16]:
idx_data

,SP500,DJIA,VIXCLS
DATE,,,
2017-02-20,NaN,NaN,NaN
2017-02-21,2365.38,20743.00,11.57
2017-02-22,2362.82,20775.60,11.74
2017-02-23,2363.81,20810.32,11.71
2017-02-24,2367.34,20821.76,11.47
...,...,...,...
2022-02-11,4418.64,34738.06,27.36
2022-02-14,4401.67,34566.17,28.33
2022-02-15,4471.07,34988.84,25.70


In [27]:
(stk_data.loc[:,("Adj Close","MSFT")]).describe()

count    1259.000000
mean        4.935759
std         0.502959
min         4.089740
25%         4.536277
50%         4.897529
75%         5.362824
max         5.835986
Name: (Adj Close, MSFT), dtype: float64

In [45]:
return_period = 5
Y = np.log(stk_data.loc[:, ('Adj Close', 'MSFT')]).diff(return_period).\
shift(-return_period)
Y.name = Y.name[-1]+'_pred'

In [38]:
Y.name=Y.name[-1]+ "_pred"

In [39]:
Y.name

'MSFT_pred'

In [57]:
X1 = np.log(stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]).diff(return_period)
X1.columns = X1.columns.droplevel()
X2 = np.log(ccy_data).diff(return_period)
X3 = np.log(idx_data).diff(return_period)
X4 = pd.concat([np.log(stk_data.loc[:, ('Adj Close', 'MSFT')]).diff(i) for i in [return_period,
                                                                             return_period*3,
                                                                             return_period*6,
                                                                             return_period*12]],axis=1).dropna()
X = pd.concat([X1,X2,X3,X4], axis=1)
dataset = pd.concat([X,Y],axis=1).dropna()

In [58]:
dataset

,IBM,GOOGL,DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,"(Adj Close, MSFT)","(Adj Close, MSFT)","(Adj Close, MSFT)","(Adj Close, MSFT)",MSFT_pred
2017-05-17,-0.002118,-0.013358,-0.022708,0.000386,-0.017912,-0.016182,0.423262,-0.021042,0.000542,0.031992,0.051037,0.018936
2017-05-18,0.000862,-0.005655,-0.026272,0.009578,-0.012067,-0.012332,0.324269,-0.005300,-0.002521,0.037984,0.056457,0.027818
2017-05-19,0.010650,-0.000513,-0.016284,0.010657,-0.003843,-0.004401,0.146429,-0.004426,-0.005596,0.035098,0.052130,0.032985
2017-05-22,0.007430,0.005043,-0.021275,0.006867,-0.003461,-0.004160,0.047784,0.006008,-0.008212,0.047025,0.063295,0.028232
2017-05-23,-0.010795,0.006139,-0.014230,0.005561,-0.000938,-0.001996,0.006551,-0.010573,-0.003271,0.052666,0.072703,0.016749
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-04,0.019511,0.071907,0.000087,0.009347,0.015378,0.010435,-0.174973,-0.007555,0.003733,-0.085354,-0.091746,-0.036278
2022-02-07,0.027105,0.028399,-0.001129,0.006970,-0.007040,-0.001160,-0.082664,-0.032784,-0.030273,-0.106261,-0.092742,-0.019969
2022-02-08,0.010934,0.012670,0.007379,0.003474,-0.005514,0.001624,-0.023964,-0.013696,0.006291,-0.117258,-0.085733,-0.013520
2022-02-09,0.003927,-0.044263,0.009400,-0.001845,-0.000479,0.003886,-0.101395,-0.007204,0.025647,-0.092147,-0.076956,-0.036288
